In [7]:
from openai import OpenAI
import pandas as pd 
import json
import duckdb
from pydantic import BaseModel, Field
from IPython.display import Markdown
import seaborn as sns

In [8]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPEN_API_KEY")
client=OpenAI(api_key=api_key)

model='gpt-4o-mini'

In [9]:
import phoenix as px
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.semconv.trace import SpanAttributes
from opentelemetry.trace import Status, StatusCode
from openinference.instrumentation import TracerProvider

In [10]:
sessions=px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [11]:
project_name="tracing-agent"

In [12]:
tracer_provider=register(
    project_name=project_name,
    endpoint = "http://localhost:6006/v1/traces",
    set_global_tracer_provider=False
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: tracing-agent
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [13]:
# OpenAIInstrumentor().instrument(tracer_provider = tracer_provider)
OpenAIInstrumentor().instrument(tracer_provider = tracer_provider)
tracer = tracer_provider.get_tracer(__name__)

In [14]:
def start_main_span(messages):
    print(f"Starting main span with messages: {messages}")

    with tracer.start_as_current_span(
        "AgentRun", openinference_span_kind="agent"
    ) as span:
        span.set_input(value=messages)
        ret = run_agent(messages)
        print(f"Main span run completed with value: {ret}")
        span.set_output(value=ret)
        span.set_status(StatusCode.OK)
        return ret

In [15]:
transaction_path='/home/toan999/coding/Agent/Store_Sales_Price_Elasticity_Promotions_Data.parquet'

In [16]:
SQL_GENERATE_PROMPT="""
Generate a SQL to access duckdb based on the prompt. Do not reply anything beside the query. 
The prompt is {prompt}.

The table name is {table_name}.
The available columns are {column}.
"""

In [17]:
def generate_sql_query(prompt: str, column: list, table_name: str) ->str:
    """Generate a SQL query based on the prompt"""
    formatted_prompt=SQL_GENERATE_PROMPT.format(prompt=prompt, column=column, table_name=table_name)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": formatted_prompt
            }
        ]
    )
    return completion.choices[0].message.content

In [18]:
@tracer.tool()
def sale_lookup(prompt):
    try:
        # db creation
        df = pd.read_parquet(transaction_path)
        table_name='sales'
        duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * from df")
        
        # query generation
        sql_generated=generate_sql_query(prompt, df.columns,table_name)
        sql_final=sql_generated.replace('```sql', '').replace('```','').strip()

        with tracer.start_as_current_span(
            "execute_sql_query",openinference_span_kind="chain"
        ) as span:
            span.set_input(value=sql_final)
        # query execution
            result=duckdb.sql(sql_final).df()
            span.set_output(value=str(result))
            span.set_status(StatusCode.OK)
        
        return result.to_string()
    except Exception as e:
        return f"Error accessing data {str(e)}"

In [19]:
exam_data=sale_lookup('Show me 20 sales of the store with highest number')
exam_data

'    Store_Number  SKU_Coded  Product_Class_Code  Sold_Date  Qty_Sold  Total_Sale_Value  On_Promo\n0           2970    6172800               22875 2021-11-21         2         37.900002         0\n1           2970    6172800               22875 2022-04-29         1         18.950001         0\n2           2970    6172800               22875 2022-07-20         1         18.950001         0\n3           2970    6172800               22875 2022-08-27         1         18.950001         0\n4           2970    6172800               22875 2022-12-17         1         18.950001         0\n5           2970    6172800               22875 2023-02-02         1         18.950001         0\n6           2970    6172800               22875 2023-03-22         4         75.800003         0\n7           2970    6172800               22875 2023-04-29         3         56.849998         0\n8           2970    6172800               22875 2023-05-07         1         18.950001         0\n9           2970   

In [20]:
analysis_prompt="""
Based on provided data {data}, help to answer the needs from {prompt}.

Let make analysis to recommend for the growth manager to take immediate action
"""

In [21]:
@tracer.tool()
def sale_analysis(prompt: str, data: str):
    """Generate data analysis prompt"""
    prompt_final=analysis_prompt.format(data=data, prompt=prompt)
    completion=client.chat.completions.create(
        model=model,
        messages=[
            {
                "role":"user",
                "content":prompt_final
            }
        ]
    )
    analysis=completion.choices[0].message.content
    return analysis if analysis else "No info here"

In [22]:
insight=sale_analysis('what is its performance?', exam_data)

In [23]:
Markdown(insight)

To analyze the provided sales data for Store 2970, focusing on the SKU Coded 6172800 and 6172850 under Product Class Code 22875, we can extract key insights and recommendations based on sales performance over time.

### Summary of Insights:

1. **Consistent Sales Volume**:
   - **SKU 6172800**: Sales are fairly consistent, with occasional spikes — for instance, the highest sales occur on 2023-03-22 (4 units) and 2023-04-29 (3 units).
   - **SKU 6172850**: It also shows stable performance with various sales volumes, but no spikes as pronounced as SKU 6172800.

2. **Sales Values**:
   - **SKU 6172800**: The total sale values range from $18.95 to $75.80, with an average sale value per sale of approximately $23.89 (calculation based on total sales divided by quantity sold).
   - **SKU 6172850**: Sales values for this SKU are similar, with total values approximately between $19.99 and $59.97. The average sale value here could be calculated similarly.

3. **Promotional Impact**:
   - **On-Promo Indicator**: None of the data points indicate items sold on promotion (On_Promo = 0). Promotional strategies could be enhanced to boost sales.

4. **Sales Trend Over Time**:
   - Sales for **SKU 6172800** show a slight upward trend in the last year with consistent sales each month, indicating a reliable product.
   - **SKU 6172850** shows steady sales but without noticeable growth.

### Recommendations for the Growth Manager:

1. **Promotional Strategies**:
   - Launch marketing campaigns or in-store promotions for both SKUs. Given that there have been no promotions so far, it's an opportunity to stimulate demand. Temporary discounts or bundle offers (like “buy one, get one at a discount”) might also encourage higher sales volumes.

2. **Monitor Seasonal Trends**:
   - Assess whether there are specific months where sales volumes spike (specifically SKU 6172800). Consider creating promotions aligned with these months to maximize sales.

3. **Increase Product Visibility**:
   - If the sales are stable but not increasing significantly, consider increasing the visibility of these products in-store and online.

4. **Customer Feedback and Review**:
   - Gather customer feedback regarding both products. Understanding customer preferences can help refine marketing and promotional strategies.

5. **Cross-Selling Opportunities**:
   - Identify complementary products that could be bundled with SKU 6172800 or 6172850. This strategy can effectively increase average transaction values and improve inventory turnover.

6. **Performance Comparison Against Competitors**:
   - Conduct a competitor analysis to see how similar products are performing at other locations. This could provide insights into price adjustments or promotional opportunities.

7. **Inventory Management**:
   - Maintain sufficient stock levels to meet potential peaks in demand, especially ahead of promotional activities. This will ensure that stockouts do not occur, leading to lost sales opportunities.

### Conclusion:
By leveraging promotional strategies, enhancing product visibility, and responding to customer feedback, the growth manager can stimulate sales growth for both SKU 6172800 and SKU 6172850, creating a stronger overall performance for store 2970. Regular analysis of sales data should continue to inform strategic decisions and optimizations in marketing efforts.

In [24]:
chart_config_prompt="""
Create chart configs to visualize following data 
{data}
The visualization goal is {visualization_goal}
"""

In [25]:
class VisualizationConfig(BaseModel):
    chart_type: str = Field(..., description="Type of chart generate")
    x_axis: str = Field(..., description="Name of the x-axis column")
    y_axis: str = Field(..., description="Name of the y-axis column")
    title: str = Field(..., description="Title of the chart")

In [26]:
@tracer.chain()
def extract_chart_config(visualization_goal:str, data:str):
    """generate config from prompt"""
    prompt_final=chart_config_prompt.format(data=data, visualization_goal=visualization_goal)

    response=client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role":"user",
                "content":prompt_final
            }
        ],
        response_format=VisualizationConfig,
    )

    try:
        result=response.choices[0].message.content

        return {
            "chart_type": result.chart_type,
            "x_axis": result.x_axis,
            "y_axis": result.y_axis,
            "title": result.title,
            "data": data
        }
    except Exception:
        return {
            "chart_type": "line",
            "x_axis": "date",
            "y_axis": "value",
            "title": visualization_goal,
            "data": data
        }

In [27]:
create_chart_prompt="""
From the follow config 
{chart_config}

Write executable python code to create a chart. Just return codes, no other text.
"""

In [28]:
@tracer.chain()
def create_chart(config:str):
    prompt_final=create_chart_prompt.format(chart_config=config)
    response=client.chat.completions.create(
        model=model,
        messages=[
            {
                "role":"user",
                "content":prompt_final
            }
        ],
    )
    result=response.choices[0].message.content
    code=result.replace('```python','').replace('```','').strip()
    return code

In [29]:
@tracer.tool()
def generate_visualization(visualization_goal, data):
    config=extract_chart_config(visualization_goal, data)
    code=create_chart(config)
    return code

In [30]:
smp_data=sale_lookup('Show me 20 sales of the store with highest number')
print(smp_data)

    Store_Number  SKU_Coded  Product_Class_Code  Sold_Date  Qty_Sold  Total_Sale_Value  On_Promo
0           2970    6172800               22875 2021-11-21         2         37.900002         0
1           2970    6172800               22875 2022-04-29         1         18.950001         0
2           2970    6172800               22875 2022-07-20         1         18.950001         0
3           2970    6172800               22875 2022-08-27         1         18.950001         0
4           2970    6172800               22875 2022-12-17         1         18.950001         0
5           2970    6172800               22875 2023-02-02         1         18.950001         0
6           2970    6172800               22875 2023-03-22         4         75.800003         0
7           2970    6172800               22875 2023-04-29         3         56.849998         0
8           2970    6172800               22875 2023-05-07         1         18.950001         0
9           2970    6172800   

In [31]:
chart_code=generate_visualization('a chart of sale by product SKU. Put the product SKU on x axis and sum of sale on the y axis', smp_data)

In [32]:
print(chart_code)

import pandas as pd
import matplotlib.pyplot as plt

# Data preparation
data = {
    'Store_Number': [2970]*16 + [2970]*5,
    'SKU_Coded': [6172800]*14 + [6172850]*5,
    'Product_Class_Code': [22875]*19,
    'Sold_Date': [
        '2021-11-21', '2022-04-29', '2022-07-20', '2022-08-27', '2022-12-17',
        '2023-02-02', '2023-03-22', '2023-04-29', '2023-05-07', '2023-08-29',
        '2023-09-30', '2023-12-08', '2023-12-14', '2024-02-10', '2024-03-14',
        '2021-11-17', '2022-01-07', '2022-06-05', '2022-06-12', '2022-10-07'
    ],
    'Qty_Sold': [2, 1, 1, 1, 1, 1, 4, 3, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1],
    'Total_Sale_Value': [
        37.900002, 18.950001, 18.950001, 18.950001, 18.950001, 18.950001,
        75.800003, 56.849998, 18.950001, 56.849998, 18.950001, 18.950001,
        18.950001, 19.990000, 19.990000, 19.990000, 59.970001, 19.990000,
        39.980000, 19.990000
    ]
}

df = pd.DataFrame(data)
df['Sold_Date'] = pd.to_datetime(df['Sold_Date'])
df['Total_Sale_Valu

In [34]:
tools = [{
    "type": "function",
    "function": {
        "name": "sale_lookup",
        "description": "Look up data from Store Sales Price Elasticity Promotions dataset",
        "parameters": {
            "type": "object",
            "properties": {
                "prompt": {
                    "type": "string",
                    "description": "The unchanged prompt users provided"
                }
            },
            "required": ["prompt"]
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "sale_analysis",
        "description": "Analyze sales data to extract insights",
        "parameters": {
            "type": "object",
            "properties": {
                "data": {"type": "string", "description": "The output from sale_lookup tool"},
                "prompt":{"type":"string", "description": "The unchanged prompt users provided"}
            },
            "required": ["data", "prompt"]
        }
    }
    },
    {
    "type": "function",
    "function": {
        "name": "generate_visualization",
        "description": "Generate Python code to create data visualizations",
        "parameters": {
            "type": "object",
            "properties": {
                "data": {"type": "string", "description": "The output from sale_lookup tool"},
                "visualization_goal":{"type":"string", "description": "The goal of visualization"}
            },
            "required": ["prompt", "visualization_goal"]
        }
    }
}]

In [35]:
tool_implementation={
    "sale_lookup": sale_lookup,
    "sale_analysis": sale_analysis,
    "generate_visualization": generate_visualization
}

In [36]:
SYSTEM_PROMPT="You are a helpfull assistant to help users understand insights from this dataset"

In [37]:
@tracer.chain()
def handle_tool_calls(tool_calls, message):
    for tool in tool_calls:
        function=tool_implementation[tool.function.name]
        args=json.loads(tool.function.arguments)
        result=function(**args)
        message.append({"role":"tool", "content":result, "tool_call_id": tool.id})
        return message

In [38]:
def run_agent(messages):
    print(f"Running agent with messages:{messages}")

    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)


    while True:
        # Router Span
        print("Making router call to OpenAI")
        with tracer.start_as_current_span(
            "router_call", openinference_span_kind='chain'
        ) as span:
            span.set_input(value=messages)
            response=client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools,
            )
    
            messages.append(response.choices[0].message)
            tool_calls=response.choices[0].message.tool_calls
            if tool_calls:
                span.set_status(StatusCode.OK)
                print(f"Received response with tool calls: {tool_calls[0].function.name} and {tool_calls[0].id}")
                print("Processing tool calls")
                messages=handle_tool_calls(tool_calls, messages)
                span.set_output(value=tool_calls)
    
            else:
                print("No tool calls, returning final response")
                span.set_output(value=response.choices[0].message.content)
                return response.choices[0].message.content

In [42]:
result = start_main_span('Show me the code for graph of sales by store in Dec 2021, and make analysis.')

Starting main span with messages: Show me the code for graph of sales by store in Dec 2021, and make analysis.
Running agent with messages:Show me the code for graph of sales by store in Dec 2021, and make analysis.
Making router call to OpenAI
Received response with tool calls: sale_lookup and call_ibYFUgLcdBXYh6lp1UgaLF3p
Processing tool calls
Making router call to OpenAI
Received response with tool calls: sale_analysis and call_1FXqC2WzeGtppZiossQnzAuw
Processing tool calls
Making router call to OpenAI
Received response with tool calls: generate_visualization and call_qeNJ9VUGC0Y4BM87Z5095Q3T
Processing tool calls
Making router call to OpenAI
No tool calls, returning final response
Main span run completed with value: Here's the analysis and code for visualizing the sales data by store for December 2021.

### Analysis Summary:
1. **Overall Sales Performance**:
   - **Total Sales Units**: 69,074 sold across all stores.
   - **Total Sales Revenue**: Approximately $900,000.
   - **Avera

In [41]:
Markdown(result)

### Analysis of Sales Data by Store in December 2021

#### Overall Sales Overview:
- **Total Quantity Sold**: 52,883 units
- **Total Sales Value**: $693,956.45
- **Average Sales Value per Store**: ~$21,035.11 
- **Average Quantity Sold per Store**: ~1,604.37
- **Average Sales per Unit**: ~$13.13/unit

#### Top and Bottom Performing Stores:
- **Top 5 Stores by Total Sales Value**:
  - Store 2904: $53,453.76
  - Store 1540: $37,569.33
  - Store 1320: $32,853.57
  - Store 3080: $32,659.61
  - Store 1650: $38,965.52
  
- **Bottom 5 Stores by Total Sales Value**:
  - Store 4400: $6,061.75
  - Store 550: $16,161.55
  - Store 4730: $18,747.21
  - Store 770: $17,978.16
  - Store 1980: $17,433.16

#### Recommendations:
- Investigate underperforming stores for potential improvement strategies.
- Learn from top-performing stores to replicate successful strategies.
- Implement targeted marketing towards lower sales stores to boost performance.

### Visualization Code:
Here is the Python code used to generate the graph displaying the sales by store in December 2021.

```python
import pandas as pd
import matplotlib.pyplot as plt

data = {
    'Store_Number': [1320, 2640, 3080, 4070, 1540, 2090, 4730, 4840, 880, 1650, 
                     550, 4180, 2200, 2310, 3630, 1870, 1760, 330, 1210, 
                     1980, 2750, 3410, 990, 770, 660, 2420, 3190, 3300, 
                     1100, 2970, 4400, 2530, 3740],
    'Total_Quantity_Sold': [2892.0, 1647.0, 2676.0, 1828.0, 3116.0, 2134.0, 
                            1654.0, 2724.0, 2353.0, 2903.0, 1301.0, 1875.0, 
                            2692.0, 2521.0, 2342.0, 2668.0, 2658.0, 1925.0, 
                            2583.0, 1776.0, 2951.0, 2227.0, 2118.0, 1684.0, 
                            2141.0, 2550.0, 2003.0, 3265.0, 2564.0, 3555.0, 
                            292.0, 1862.0, 2244.0],
    'Total_Sales_Value': [32853.569915, 20872.25, 32659.60996, 21321.059979, 
                          37569.329945, 23003.409946, 18747.209966, 29933.489884, 
                          31668.100037, 38965.520022, 16161.550002, 21071.019946, 
                          34780.07002, 28763.669892, 29063.319957, 30694.519899, 
                          30765.979889, 21555.029918, 34413.949924, 17433.159965, 
                          35648.789999, 26714.789922, 24713.900014, 17978.159951, 
                          22993.589873, 26545.169849, 24083.250015, 39474.609946, 
                          33196.209952, 53453.759857, 6061.750041, 20844.890021, 
                          26977.429915]
}

df = pd.DataFrame(data)
plt.figure(figsize=(10, 5))
plt.plot(df['Store_Number'], df['Total_Sales_Value'], marker='o')
plt.title('Sales by Store in December 2021')
plt.xlabel('Store Number')
plt.ylabel('Total Sales Value')
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()
```

This code creates a line graph that illustrates the total sales by each store, providing a clear visualization of performance across the board. If you need further assistance or modifications to the analysis or the visualization, feel free to ask!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = {
    'Store_Number': [1540, 4840, 3080, 1320, 4070, 4730, 2090, 2640, 330, 1210, 1760, 2750, 1980, 4400, 2530, 3740, 2970, 1650, 880, 4180, 550, 3630, 2200, 1870, 2310, 3410, 990, 660, 2420, 770, 1100, 3190, 3300],
    'Total_Qty_Sold': [3116.0, 2724.0, 2676.0, 2892.0, 1828.0, 1654.0, 2134.0, 1647.0, 1925.0, 2583.0, 2658.0, 2951.0, 1776.0, 292.0, 1862.0, 2244.0, 3555.0, 2903.0, 2353.0, 1875.0, 1301.0, 2342.0, 2692.0, 2668.0, 2521.0, 2227.0, 2118.0, 2141.0, 2550.0, 1684.0, 2564.0, 2003.0, 3265.0],
    'Total_Sale_Value': [37569.329945, 29933.489884, 32659.609960, 32853.569915, 21321.059979, 18747.209966, 23003.409946, 20872.250000, 21555.029918, 34413.949924, 30765.979889, 35648.789999, 17433.159965, 6061.750041, 20844.890021, 26977.429915, 53453.759857, 38965.520022, 31668.100037, 21071.019946, 16161.550002, 29063.319957, 34780.070020, 30694.519899, 28763.669892, 26714.789922, 24713.900014, 22993.589873, 26545.169849, 17978.159951, 33196.209952, 24083.250015, 39474.609946]
}

df = pd.DataFrame(data)

plt.figure(figsize=(12, 6))
plt.bar(df['Store_Number'].astype(str), df['Total_Sale_Value'], color='skyblue')
plt.title('Sales by Store for December 2021')
plt.xlabel('Store Number')
plt.ylabel('Total Sale Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
result = start_main_span([{"role": "user", 
                           "content": "Which stores did the best in 2021?"}])